# Results of DDNet Trained on CIFAR10

In this notebook I display results in terms of accuracy on test data for DDNet models trained on the CIFAR10 dataset using various regularization techniques. The DDNet model and the CIFAR10 dataset were set up as in Hoffman 2019 (for all the models). The CIFAR10 data is preprocessed by normalizing using mean [0.5, 0.5, 0.5] and variance [0.5, 0.5, 0.5]. The batch size is 100. The model optimizes using SGD with momentum p = 0.9, and standard cross-entropy loss. Model parameters are initialized using Glorot initialization (See Glorot & Bengio 2010), expect for SVB regularization which uses orthogonal initialization. Models are trained with no regularization, L2 regularization, SVB regularization and Jacobian regularization, both with and without dropout with a dropout rate of p_drop = 0.5. The L2 regularization coefficient and Jacobian regularization coefficient are the same as in Hoffman 2019: l2_lmbd = 0.0005 and lambda_jacobian_reg = 0.01. For SVB regularization I use the hyperparameters svb_freq=600 and svb_eps = 0.05. The learning rate starts at 0.1, and is reduced to 0.01 and 0.001 1/3 and 2/3s into training, respectively. The models are trained for 250 epochs.

### Imports and Model Loading

In [ ]:
import jupyter_black
import numpy as np
import pandas as pd
import torch
from scipy import stats
from torchsummary import summary
from tqdm import tqdm

from data_generators import data_loader_CIFAR10
from model_classes import DDNet
from plotting_tools import get_random_img, generate_random_vectors
from tools import accuracy, compute_total_variation, ModelInfo

jupyter_black.load()

In [ ]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load MNIST data
train_loader, test_loader = data_loader_CIFAR10()

# Summary of model
summary_model = DDNet().to(device)
summary(summary_model, (3, 32, 32))

In [ ]:
# Load models
dataset = "cifar10"
model_names = [
    "model_no_reg_0",
    "model_no_reg_no_dropout_0",
    "model_l2_0",
    "model_l2_no_dropout_0",
    "model_jacobi_0",
    "model_jacobi_no_dropout_0",
    "model_svb_0",
    "model_svb_no_dropout_0",
    "model_jacobi_and_svb_0",
    "model_all_0",
    "model_jacobi_and_l2_0",
    "model_svb_and_l2_0",
]

models = {name: ModelInfo(name, dataset) for name in model_names}

### Accuracies

Here I calculate the accuracies of each model as a 95% confidence interval. I have trained five models of each variation that I use to calculate the CI.

In [ ]:
# Dictionary to hold accuracy data
accuracy_data = {}

# Calculate accuracies with 95% CI for models
for t in model_types:
    for v in model_versions:
        accuracies = []
        for i in range(5):  # Assuming 5 versions of each model
            model_name = f"model_{t}{v}_{i}"
            model = models[model_name].model
            acc = accuracy(model, test_loader, device)
            accuracies.append(acc)

        # Calculate mean accuracy
        mean_accuracy = np.mean(accuracies)

        # Calculate standard error
        std_error = stats.sem(accuracies)

        # Calculate confidence interval
        CI = std_error * stats.t.ppf((1 + 0.95) / 2, len(accuracies) - 1)

        # Store results in the data dictionary
        accuracy_data[f"{t}{v if v != '' else '_dropout'}"] = (mean_accuracy, CI)

In [ ]:
# Convert the dictionary into a DataFrame
df = pd.DataFrame.from_dict(
    accuracy_data, orient="index", columns=["Mean Accuracy", "Confidence Interval"]
)

# Convert the numbers from decimal to percentage and round to four decimal places
df = df.multiply(100).round(3)

# Create a new column that combines Mean Accuracy and Confidence Interval
df["Accuracy +/- CI (95%)"] = df.apply(
    lambda row: f"{row['Mean Accuracy']} +/- {row['Confidence Interval']}", axis=1
)

# Drop the original columns
df = df.drop(columns=["Mean Accuracy", "Confidence Interval"])

# Reset the index
df = df.reset_index().rename(columns={"index": "Model"})

display(df)

### Total Variation

Total variation is a measure for roughness/complexity in images. I generate 25 different images for each model to get a good mean and standard deviation, and give the results as a tabel for each model with the eight models (models with and without dropout) as rows, and the three different zoom levels as columns. The tabel contains the mean and 95 % confidence interval for the total variation for each model at each zoom level, for both isotropic and anisotropic total variation.

#### Isotropic Total Variation

In [ ]:
# Define zoom levels, number of images, and confidence level for confidence interval calculation
zoom_levels = [0.025, 0.01, 0.001]
n_images = 10
confidence_level = 0.95

# Select only the first model of each type (those with index 0)
selected_models = [name for name in model_names if name.split("_")[-1] == "0"]

# Dataframe to store results
cols = pd.MultiIndex.from_product([zoom_levels, ["mean", "conf_interval"]])
df_results_isotropic = pd.DataFrame(index=selected_models, columns=cols)

# Loop over the selected models
for name in tqdm(selected_models):
    model = models[name].model  # Get model
    model.to(device)
    tv_values = {
        zoom: [] for zoom in zoom_levels
    }  # To store total variation values for each zoom level

    # Generate tv values for n_images number of images
    for _ in range(n_images):
        # Get random image and vectors
        img = get_random_img(test_loader)
        v1, v2 = generate_random_vectors(img)

        # Compute the isotropic total variation of decision boundaries for the current model
        # and the generated image at different zoom levels
        tv_list = compute_total_variation(
            model,
            img,
            v1,
            v2,
            device,
            resolution=300,
            zoom=zoom_levels,
            mode="isotropic",
        )
        for zoom, tv in zip(zoom_levels, tv_list):
            tv_values[zoom].append(tv)

    # Calculate mean and 95% confidence interval for total variation values at each zoom level
    for zoom in zoom_levels:
        mean = np.mean(tv_values[zoom])
        std_err = np.std(tv_values[zoom]) / np.sqrt(n_images)
        conf_interval = stats.t.ppf((1 + confidence_level) / 2, n_images - 1) * std_err
        df_results_isotropic.loc[name, (zoom, "mean")] = mean
        df_results_isotropic.loc[name, (zoom, "conf_interval")] = conf_interval

In [ ]:
print("Isotropic Total variation for three different zoom levels (of images generated)")
display(df_results_isotropic)

#### Anisotropic Total Variation

In [ ]:
# Define zoom levels, number of images, and confidence level for confidence interval calculation
zoom_levels = [0.025, 0.01, 0.001]
n_images = 10
confidence_level = 0.95

# Select only the first model of each type (those with index 0)
selected_models = [name for name in model_names if name.split("_")[-1] == "0"]

# Dataframe to store results
cols = pd.MultiIndex.from_product([zoom_levels, ["mean", "conf_interval"]])
df_results_anisotropic = pd.DataFrame(index=selected_models, columns=cols)

# Loop over the selected models
for name in tqdm(selected_models):
    model = models[name].model  # Get model
    model.to(device)
    tv_values = {
        zoom: [] for zoom in zoom_levels
    }  # To store total variation values for each zoom level

    # Generate tv values for n_images number of images
    for _ in range(n_images):
        # Get random image and vectors
        img = get_random_img(test_loader)
        v1, v2 = generate_random_vectors(img)

        # Compute the isotropic total variation of decision boundaries for the current model
        # and the generated image at different zoom levels
        tv_list = compute_total_variation(
            model,
            img,
            v1,
            v2,
            device,
            resolution=300,
            zoom=zoom_levels,
            mode="anisotropic",
        )
        for zoom, tv in zip(zoom_levels, tv_list):
            tv_values[zoom].append(tv)

    # Calculate mean and 95% confidence interval for total variation values at each zoom level
    for zoom in zoom_levels:
        mean = np.mean(tv_values[zoom])
        std_err = np.std(tv_values[zoom]) / np.sqrt(n_images)
        conf_interval = stats.t.ppf((1 + confidence_level) / 2, n_images - 1) * std_err
        df_results_anisotropic.loc[name, (zoom, "mean")] = mean
        df_results_anisotropic.loc[name, (zoom, "conf_interval")] = conf_interval

In [ ]:
print(
    "Anisotropic Total variation for three different zoom levels (of images generated)"
)
display(df_results_anisotropic)